# ISIC 2019 皮肤病变分类

本笔记本使用ResNet50模型对ISIC 2019数据集中的皮肤病变图像进行分类。

## 数据集包含以下类别：
- MEL: 黑色素瘤
- NV: 黑素细胞痣
- BCC: 基底细胞癌
- AK: 光化性角化病
- BKL: 良性角化病样病变
- DF: 皮肤纤维瘤
- VASC: 血管病变
- SCC: 鳞状细胞癌

In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import numpy as np
 
# 1. 定义 ISIC2019Dataset 类
class ISIC2019Dataset(Dataset):
    def __init__(self, images_dir, labels_file, transform=None):
        """
        初始化 ISIC 2019 数据集。
        参数:
            images_dir (str): 图像文件夹的路径
            labels_file (str): 标签文件的路径 (CSV 格式)
            transform (callable, optional): 对图像进行的变换
        """
        self.images_dir = images_dir
        self.labels_df = pd.read_csv(labels_file)
        
        # 忽略最后一列 "UNK"
        self.labels_df = self.labels_df.iloc[:, :-1]
        self.transform = transform
        self.labels = self.labels_df.iloc[:, 1:].values.astype('float')
        
        # 统计图片文件夹中的总图像数
        self.total_images_in_dir = len(os.listdir(images_dir))
        # 统计标签文件中的记录数
        self.total_images_in_csv = len(self.labels_df)
 
    def __len__(self):
        """
        返回数据集的大小。
        """
        return len(self.labels_df)
 
    def __getitem__(self, idx):
        """
        根据索引获取数据。
        参数:
            idx (int): 索引
        返回:
            image (Tensor): 变换后的图像
            label (Tensor): 标签
        """
        # 获取图像路径
        img_name = os.path.join(self.images_dir, self.labels_df.iloc[idx, 0] + '.jpg')
        image = Image.open(img_name).convert('RGB')
        
        # 获取标签
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
 
        # 应用变换
        if self.transform:
            image = self.transform(image)
        
        return image, label
    
    def get_class_distribution(self):
        """
        计算数据集中每个类别的样本数量
        """
        class_counts = self.labels_df.iloc[:, 1:].sum(axis=0)
        return class_counts
 
    def print_image_counts(self):
        """
        打印图片文件夹中的总图像数和标签文件中的记录数
        """
        print(f"图片文件夹中的图像总数: {self.total_images_in_dir}")
        print(f"标签文件中的图像总数: {self.total_images_in_csv}")
 
# 2. 设置图像转换
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小
    transforms.ToTensor(),  # 转换为 Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # 标准化
])
 
# 3. 定义图像和标签的路径
images_dir = '../data/ISIC_2019_Training_Input'  # 替换为实际图像路径
labels_file = '../data/ISIC_2019_Training_GroundTruth.csv'  # 替换为实际标签文件路径
 
# 4. 创建数据集实例
isic_dataset = ISIC2019Dataset(images_dir=images_dir, labels_file=labels_file, transform=data_transforms)
 
# 打印图片和标签的数量
isic_dataset.print_image_counts()
 
# 5. 根据 80%/20% 的比例划分数据集
train_size = int(0.8 * len(isic_dataset))  # 80% 作为训练集
test_size = len(isic_dataset) - train_size  # 20% 作为测试集
 
train_dataset, test_dataset = random_split(isic_dataset, [train_size, test_size])
 
# 5. 打印整个数据集的类别分布
class_distribution = isic_dataset.get_class_distribution()
print("数据集中每个类别的图像数量:")
print(class_distribution)
 
# 6. 统计并打印训练集和测试集的类别分布
def calculate_class_distribution(dataset, dataset_indices, num_classes):
    labels = np.array([dataset.labels[idx] for idx in dataset_indices])
    class_counts = labels.sum(axis=0)
    return class_counts
 
num_classes = isic_dataset.labels.shape[1]  # 获取类别数量
train_class_distribution = calculate_class_distribution(isic_dataset, train_dataset.indices, num_classes)
test_class_distribution = calculate_class_distribution(isic_dataset, test_dataset.indices, num_classes)
 
print("训练集中每个类别的图像数量:")
print(train_class_distribution)
 
print("测试集中每个类别的图像数量:")
print(test_class_distribution)
 
# 7. 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
 

图片文件夹中的图像总数: 25333
标签文件中的图像总数: 25331
数据集中每个类别的图像数量:
MEL      4522.0
NV      12875.0
BCC      3323.0
AK        867.0
BKL      2624.0
DF        239.0
VASC      253.0
SCC       628.0
dtype: float64
训练集中每个类别的图像数量:
[ 3601. 10298.  2680.   686.  2099.   180.   208.   512.]
测试集中每个类别的图像数量:
[ 921. 2577.  643.  181.  525.   59.   45.  116.]


## 定义ResNet50模型

使用预训练的ResNet50模型，并修改最后的全连接层以适应我们的多标签分类任务。

In [ ]:
import torch.nn as nn
import torchvision.models as models
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tqdm import tqdm
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

# 设置设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 定义ResNet50模型
class SkinLesionModel(nn.Module):
    def __init__(self, num_classes=8):
        super(SkinLesionModel, self).__init__()
        # 加载预训练的ResNet50模型
        self.resnet = models.resnet50(pretrained=True)
        
        # 冻结所有层
        for param in self.resnet.parameters():
            param.requires_grad = False
            
        # 替换最后的全连接层
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Linear(num_features, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes),
            nn.Sigmoid()  # 使用Sigmoid激活函数用于多标签分类
        )
        
    def forward(self, x):
        return self.resnet(x)

# 创建模型实例
model = SkinLesionModel(num_classes=num_classes)
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.BCELoss()  # 二元交叉熵损失函数，适用于多标签分类
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

print(f"模型已创建，类别数量: {num_classes}")

## 训练模型

训练ResNet50模型进行皮肤病变分类。

In [ ]:
# 训练函数
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    train_losses = []
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 清零梯度
            optimizer.zero_grad()
            
            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # 反向传播和优化
            loss.backward()
            optimizer.step()
            
            # 统计
            running_loss += loss.item() * inputs.size(0)
            progress_bar.set_postfix({'loss': loss.item()})
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        
        # 更新学习率
        scheduler.step(epoch_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')
    
    return train_losses

# 评估函数
def evaluate_model(model, test_loader, criterion, threshold=0.5):
    model.eval()
    test_loss = 0.0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # 前向传播
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # 统计
            test_loss += loss.item() * inputs.size(0)
            
            # 将输出转换为二进制预测
            preds = (outputs > threshold).float().cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels.cpu().numpy())
    
    test_loss = test_loss / len(test_loader.dataset)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    return test_loss, all_preds, all_labels

# 训练模型
num_epochs = 10
train_losses = train_model(model, train_loader, criterion, optimizer, num_epochs=num_epochs)

# 绘制训练损失曲线
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs+1), train_losses, marker='o')
plt.title('训练损失')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()

## 评估模型

在测试集上评估模型性能，并可视化结果。

In [ ]:
# 评估模型
test_loss, all_preds, all_labels = evaluate_model(model, test_loader, criterion)
print(f'测试损失: {test_loss:.4f}')

# 计算每个类别的准确率
class_names = class_distribution.index.tolist()
accuracies = []

for i in range(num_classes):
    acc = accuracy_score(all_labels[:, i], all_preds[:, i])
    accuracies.append(acc)
    print(f'{class_names[i]} 准确率: {acc:.4f}')

# 计算总体准确率
overall_acc = np.mean(accuracies)
print(f'总体准确率: {overall_acc:.4f}')

# 绘制每个类别的准确率
plt.figure(figsize=(12, 6))
plt.bar(class_names, accuracies)
plt.title('各类别准确率')
plt.xlabel('类别')
plt.ylabel('准确率')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# 为每个类别绘制混淆矩阵
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
axes = axes.flatten()

for i in range(num_classes):
    cm = confusion_matrix(all_labels[:, i], all_preds[:, i])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i])
    axes[i].set_title(f'{class_names[i]}')
    axes[i].set_xlabel('预测')
    axes[i].set_ylabel('真实')
    axes[i].set_xticklabels(['阴性', '阳性'])
    axes[i].set_yticklabels(['阴性', '阳性'])

plt.tight_layout()
plt.show()

## 可视化预测结果

随机选择一些测试图像，并显示模型的预测结果。

In [ ]:
import random

# 可视化预测结果
def visualize_predictions(model, test_dataset, class_names, num_samples=5, threshold=0.5):
    model.eval()
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 3*num_samples))
    
    # 随机选择样本
    indices = random.sample(range(len(test_dataset)), num_samples)
    
    for i, idx in enumerate(indices):
        # 获取图像和标签
        image, label = test_dataset[idx]
        
        # 预测
        with torch.no_grad():
            input_tensor = image.unsqueeze(0).to(device)
            output = model(input_tensor)
            pred = (output > threshold).float().cpu().numpy()[0]
        
        # 转换图像用于显示
        img = image.cpu().numpy().transpose((1, 2, 0))
        mean = np.array([0.485, 0.456, 0.406])
        std = np.array([0.229, 0.224, 0.225])
        img = std * img + mean
        img = np.clip(img, 0, 1)
        
        # 显示图像
        axes[i, 0].imshow(img)
        axes[i, 0].set_title('原始图像')
        axes[i, 0].axis('off')
        
        # 显示真实标签和预测标签
        true_classes = [class_names[j] for j in range(len(class_names)) if label[j] > 0.5]
        pred_classes = [class_names[j] for j in range(len(class_names)) if pred[j] > 0.5]
        
        axes[i, 1].axis('off')
        axes[i, 1].text(0.1, 0.7, f'真实标签: {", ".join(true_classes)}', fontsize=12)
        axes[i, 1].text(0.1, 0.3, f'预测标签: {", ".join(pred_classes)}', fontsize=12)
        
        # 设置颜色：绿色表示正确，红色表示错误
        if set(true_classes) == set(pred_classes):
            axes[i, 1].text(0.1, 0.1, '预测正确', fontsize=12, color='green')
        else:
            axes[i, 1].text(0.1, 0.1, '预测错误', fontsize=12, color='red')
    
    plt.tight_layout()
    plt.show()

# 可视化一些预测结果
visualize_predictions(model, test_dataset, class_names, num_samples=5)

## 保存模型

保存训练好的模型，以便将来使用。

In [ ]:
# 保存模型
torch.save(model.state_dict(), 'isic2019_resnet50_model.pth')
print('模型已保存到 isic2019_resnet50_model.pth')

# 如何加载模型
'''
# 加载模型
loaded_model = SkinLesionModel(num_classes=num_classes)
loaded_model.load_state_dict(torch.load('isic2019_resnet50_model.pth'))
loaded_model = loaded_model.to(device)
loaded_model.eval()
'''